## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Medical LLM - 10B](https://aws.amazon.com/marketplace/pp/prodview-x3uprn5edkwdq)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

- **Model**: `JSL-Medical-LLM-10B`  
- **Model Description**: Medical LLM built for advanced clinical reasoning, complex terminology processing, and detailed healthcare documentation analysis. Delivers improved accuracy on specialized medical tasks while remaining computationally efficient, making it well-suited for retrieval-augmented generation (RAG) use cases in healthcare and life sciences.


In [5]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [3]:
import os
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [6]:
model_name = "JSL-Medical-LLM-10B"

real_time_inference_instance_type = "ml.g4dn.12xlarge"
batch_transform_inference_instance_type = "ml.g4dn.12xlarge"

## 2. Create a deployable model from the model package.

In [7]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn, 
    sagemaker_session=sagemaker_session, 
)

## Model Configuration Documentation  

### Default Configuration  
The container comes with the following default configurations:  

| Parameter                  | Default Value | Description                                                                   |  
|----------------------------|---------------|-------------------------------------------------------------------------------|  
| **`dtype`**                | `float16`     | Data type for model weights and activations                                   |  
| **`max_model_len`**        | `32,768`      | Maximum length for input and output combined (`input + output ≤ max_model_len`) |  
| **`tensor_parallel_size`** | Auto          | Automatically set to the number of available GPUs                            |  
| **`host`**                 | `0.0.0.0`     | Host name                                                                     |  
| **`port`**                 | `8080`        | Port number                                                                   |  

### Hardcoded Settings  
The following settings are hardcoded in the container and cannot be changed:  

| Parameter       | Value           | Description                           |  
|-----------------|-----------------|---------------------------------------|  
| **`model`**     | `/opt/ml/model` | Model path where SageMaker mounts the model |  

### Configurable Environment Variables  
You can customize the vLLM server by setting environment variables when creating the model.  

**Any parameter from the [vLLM documentation](https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html#vllm-serve) can be set using the corresponding environment variable with the `SM_VLLM_` prefix.**  

The container uses a script similar to the [SageMaker entrypoint example](https://docs.vllm.ai/en/latest/examples/sagemaker_entrypoint.html) from the vLLM documentation to convert environment variables to command-line arguments.  

---  

## Input Format  

### 1. Chat Completion  

#### Example Payload  
```json  
{  
    "model": "/opt/ml/model",  
    "messages": [  
        {"role": "system", "content": "You are a helpful medical assistant."},  
        {"role": "user", "content": "What should I do if I have a fever and body aches?"}  
    ],  
    "max_tokens": 1024,  
    "temperature": 0.7  
}  
```  

For additional parameters:  
- [ChatCompletionRequest](https://github.com/vllm-project/vllm/blob/v0.7.3/vllm/entrypoints/openai/protocol.py#L212)  
- [OpenAI's Chat API](https://platform.openai.com/docs/api-reference/chat/create)  

---  

### 2. Text Completion  

#### Single Prompt Example  
```json  
{  
    "model": "/opt/ml/model",  
    "prompt": "How can I maintain good kidney health?",  
    "max_tokens": 512,  
    "temperature": 0.6  
}  
```  

#### Multiple Prompts Example  
```json  
{  
    "model": "/opt/ml/model",  
    "prompt": [  
        "How can I maintain good kidney health?",  
        "What are the best practices for kidney care?"  
    ],  
    "max_tokens": 512,  
    "temperature": 0.6  
}  
```  

Reference:  
- [CompletionRequest](https://github.com/vllm-project/vllm/blob/v0.7.3/vllm/entrypoints/openai/protocol.py#L642)  
- [OpenAI's Completions API](https://platform.openai.com/docs/api-reference/completions/create)  

---  

### Important Notes:
- **Streaming Responses:** Add `"stream": true` to your request payload to enable streaming
- **Model Path Requirement:** Always set `"model": "/opt/ml/model"` (SageMaker's fixed model location)

## 3. Create an SageMaker Endpoint

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
    model_data_download_timeout=3600
)

### 3.1 Real-time inference via Amazon SageMaker Endpoint

#### Initial setup

In [10]:
prompt1 = "How do emerging mRNA technologies compare to traditional vaccine approaches for disease prevention?"

prompt2 = """A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus.

Which of the following is the best treatment for this patient?
A: Ampicillin
B: Ceftriaxone
C: Ciprofloxacin
D: Doxycycline
E: Nitrofurantoin
"""

prompts = [
    "What are the early warning signs of stroke and what should I do if I suspect someone is having one?",
    "How do different classes of antidepressants work and what factors determine which medication might be prescribed?",
    "What is the relationship between inflammation, autoimmune conditions, and chronic disease progression?"
]

In [11]:
system_prompt = "You are a helpful medical assistant."

In [13]:
def invoke_realtime_endpoint(record):

    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType="application/json",
        Accept="application/json",
        Body=json.dumps(record),
    )

    return json.load(response["Body"])

#### Chat Completion

In [14]:
input_data = {
    "model": "/opt/ml/model",
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt1},
    ],
    "max_tokens": 2048,
    "temperature": 0.8,
    "top_p": 0.95,
}

result = invoke_realtime_endpoint(input_data)
output_content = result['choices'][0]['message']['content']
print(output_content)

Emerging mRNA technologies and traditional vaccine approaches have different mechanisms of action and advantages for disease prevention.

Traditional Vaccines:

1. Live-attenuated vaccines: These vaccines contain weakened or dead pathogens that stimulate an immune response without causing the disease. Examples include the measles, mumps, and rubella (MMR) vaccine and the oral poliovirus vaccine.
2. Inactivated vaccines: These vaccines contain whole, killed pathogens that cannot replicate but still trigger an immune response. Examples include the polio, hepatitis A, and rabies vaccines.
3. Subunit, recombinant, and conjugate vaccines: These vaccines contain specific parts of the pathogen, such as proteins or sugars, that stimulate an immune response without introducing the whole pathogen. Examples include the human papillomavirus (HPV) vaccine and the pneumococcal conjugate vaccine.

Advantages:

1. Well-established safety profile: Traditional vaccines have been used for decades, and th

#### Text Completion

In [16]:
input_data ={
        "model": "/opt/ml/model",
        "prompt": f"{system_prompt}\n\nUser: {prompt2}\n\nAssistant:",
        "max_tokens": 2048,
        "temperature": 0.8,
        "top_p": 0.95,
    }

result = invoke_realtime_endpoint(input_data)
output_text = result['choices'][0]['text']
print(output_text)

 The answer is E. Nitrofurantoin

In a pregnant woman, burning upon urination with no fever or other symptoms is most likely a urinary tract infection (UTI). Nitrofurantoin is the safest antibiotic to use in pregnancy, especially during the second and third trimesters. Ampicillin, Ceftriaxone, Ciprofloxacin, and Doxycycline are either contraindicated or should be avoided in pregnancy due to potential risks to the fetus. It is important to treat UTI promptly in pregnancy to prevent complications such as pyelonephritis and preterm labor.


### 3.2 Real-time inference response as a stream via Amazon SageMaker Endpoint

In [18]:
def invoke_streaming_endpoint(record):
    try:
        response = sm_runtime.invoke_endpoint_with_response_stream(
            EndpointName=model_name,
            Body=json.dumps(record),
            ContentType="application/json",
            Accept="text/event-stream"
        )

        for event in response["Body"]:
            if "PayloadPart" in event:
                chunk = event["PayloadPart"]["Bytes"].decode("utf-8")
                if chunk.startswith("data:"):
                    try:
                        data = json.loads(chunk[5:].strip())
                        if "choices" in data and len(data["choices"]) > 0:
                            choice = data["choices"][0]
                            if "text" in choice:
                                yield choice["text"]
                            elif "delta" in choice and "content" in choice["delta"]:
                                yield choice["delta"]["content"]

                    except json.JSONDecodeError:
                        continue 
            elif "ModelStreamError" in event:
                error = event["ModelStreamError"]
                yield f"\nStream error: {error['Message']} (Error code: {error['ErrorCode']})"
                break
            elif "InternalStreamFailure" in event:
                failure = event["InternalStreamFailure"]
                yield f"\nInternal stream failure: {failure['Message']}"
                break
    except Exception as e:
        yield f"\nAn error occurred during streaming: {str(e)}"

#### Chat Completion

In [19]:
payload = {
    "model": "/opt/ml/model",
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt1}
    ],
    "max_tokens": 2048,
    "temperature": 0.8,
    "top_p": 0.95,
    "stream": True
}

for chunk in invoke_streaming_endpoint(payload):
    print(chunk, end="", flush=True)

Emerging mRNA messenger RNA) technologies and traditional vaccine approaches both aim to prevent diseases, but they differ significantly in their methodologies, mechanisms of action, and implications for public health. Here's a comparison of the two:

1. Mechanisms of Action:
   - Traditional Vaccines: Most traditional vaccines are based on attenuated or inactivated pathogens (e.g., live-attenuated, inactivated, subunit), or they use purified or recombinant antigens (e.g., protein subunit vaccines). These vaccines introduce weakened or killed pathogens or their components into the body, stimulating the immune system to recognize and respond to the pathogen.
   - mRNA Vaccines: mRNA vaccines, such as those used in the COVID-19 vaccines, work by delivering a small piece of genetic material (messenger RNA) that instructs cells to produce a specific protein—often a viral antigen. This protein is then recognized by the immune system, leading to the production of antibodies and the developme

#### Text Completion

In [20]:
payload = {
    "model": "/opt/ml/model",
    "prompt": f"{system_prompt}\n\nUser: {prompt2}\n\nAssistant:",
    "max_tokens": 2048,
    "temperature": 0.8,
    "top_p": 0.95,
    "stream": True
}

for chunk in invoke_streaming_endpoint(payload):
    print(chunk, end="", flush=True)

 E: Nitrofurantoin

Based on the clinical presentation and the absence of risk factors for UTI in, the most likely diagnosis is a urinary tract infection (UTI). Nitrofurantoin is the recommended antibiotic for treating UTIs in due to its safety profile and effectiveness in treating lower urinary tract infections.

Ampicillin (A), Ceftriaxone (B), Ciprofloxacin (C), and Doxycycline (D) are not the antibiotics for treating UTIs in. Ampicillin and Ceftriaxone may be used for complicated UTIs, but Nitrofurantoin is generally preferred for uncomplicated cases. Ciprofloxacin and Doxycycline are contraindicated in due to potential teratogenic effects.

Therefore, Nitrofurantoin (E) is the best treatment option for this patient.

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [24]:
validation_json_file_name1 = "input1.json"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/"


def write_and_upload_to_s3(input_data, file_name):
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_name}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [25]:
input_json_data1 = json.dumps(
    {
        "model": "/opt/ml/model",
        "prompt": [f"{system_prompt}\n\nUser: {prompt}\n\nAssistant:" for prompt in prompts],
        "max_tokens": 2048,
        "temperature": 0.8,
        "top_p": 0.95,
    }
)

write_and_upload_to_s3(input_json_data1, f"{validation_json_file_name1}")

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path,
)
transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
from urllib.parse import urlparse

def retrieve_json_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)
    result = json.loads(response["Body"].read().decode("utf-8"))
    
    for idx, choice in enumerate(result.get("choices", [])):
        print(f"Response {idx + 1}:\n{choice.get('text', '')}\n{'=' * 75}")

In [30]:
retrieve_json_output_from_s3(validation_json_file_name1)

Response 1:
 The early warning signs of a stroke include sudden weakness or numbness of the face, arm, or leg, especially on one side of the body; sudden confusion, trouble speaking, or understanding speech; sudden trouble seeing in one or both eyes; sudden trouble walking, dizziness, or loss of balance or coordination; and sudden severe headache with no known cause. If you suspect someone is having a stroke, call emergency services immediately and ask for an ambulance. While waiting for help, try to keep the person calm, comfortable, and on their side with their head slightly tilted back to prevent choking.

Is there anything else you would like to know about stroke symptoms or first aid?
Response 2:
 Antidepressants are a class of drugs used to treat depression and other mood disorders. There are several different classes of antidepressants, and each works in a slightly different way to help improve mood and reduce symptoms. The five main classes of antidepressants are:

1. Selective

Congratulations! You just verified that the batch transform job is working as expected. Since the model is not required, you can delete it. Note that you are deleting the deployable model. Not the model package.

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

